In [5]:
import pandas as pd 
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [9]:
os.getcwd()
df=pd.read_csv("dataset/02-14-2018.csv")

In [11]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [13]:
df.tail()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1048570,80,6,14/02/2018 10:53:23,10156986,5,5,1089,1923,587,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1048571,80,6,14/02/2018 10:53:33,117,2,0,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1048572,80,6,14/02/2018 10:53:28,5095331,3,1,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1048573,80,6,14/02/2018 10:53:28,5235511,3,1,0,0,0,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
1048574,443,6,14/02/2018 10:53:28,5807256,6,4,327,145,245,0,...,20,291569.0,0.0,291569,291569,5515650.0,0.0,5515650,5515650,Benign


In [15]:
df.shape

(1048575, 80)

In [17]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

In [21]:
df.describe()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,...,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,4.876262e+03,8.107557e+00,6.255555e+06,6.206622e+00,7.211191e+00,4.479936e+02,4.521803e+03,1.745736e+02,8.389535e+00,3.879579e+01,...,2.793536e+00,2.327970e+01,5.152449e+04,2.136151e+04,8.789157e+04,3.995477e+04,3.101206e+06,7.297218e+05,4.812391e+06,2.126920e+06
std,1.444344e+04,4.460625e+00,1.260291e+09,4.447851e+01,1.048682e+02,1.573541e+04,1.515021e+05,2.876713e+02,1.948279e+01,5.331882e+01,...,5.557106e+00,1.106185e+01,5.815586e+05,2.186405e+05,7.395725e+05,5.602693e+05,5.414780e+08,3.820031e+08,1.522117e+09,1.817013e+07
min,0.000000e+00,0.000000e+00,-9.190110e+11,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.200000e+01,6.000000e+00,7.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.300000e+01,6.000000e+00,1.023000e+03,2.000000e+00,1.000000e+00,3.600000e+01,5.500000e+01,3.400000e+01,0.000000e+00,2.566667e+01,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,6.000000e+00,4.066690e+05,7.000000e+00,6.000000e+00,4.550000e+02,7.680000e+02,1.990000e+02,0.000000e+00,5.550000e+01,...,4.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553300e+04,1.700000e+01,1.200000e+08,5.115000e+03,9.198000e+03,8.591554e+06,1.339773e+07,6.444000e+04,1.460000e+03,1.121703e+04,...,1.031000e+03,4.800000e+01,1.102401e+08,5.723446e+07,1.102401e+08,1.102401e+08,3.394503e+11,2.432682e+11,9.797810e+11,1.260300e+10


In [23]:
df['Label'].value_counts()

Label
Benign            667626
FTP-BruteForce    193360
SSH-Bruteforce    187589
Name: count, dtype: int64

In [25]:
df.isna().sum().to_numpy()


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 2277,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int64)

In [27]:
dfCleaned = df.dropna()
dfCleaned.isna().sum().to_numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [31]:
df.drop(columns=["Timestamp"], inplace=True)
df.drop(columns=["Protocol"], inplace=True)

In [35]:
label_encoder = LabelEncoder()
df['Label']= label_encoder.fit_transform(df['Label'])
print(df['Label'].unique())
print(df['Label'].value_counts())

[0 1 2]
Label
0    667626
1    193360
2    187589
Name: count, dtype: int64


In [41]:
corr_matrix = df.corr().abs()

upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)
high_corr_features = [column for column in upper.columns if any(upper[column] > 0.9)]

high_corr_features

['TotLen Bwd Pkts',
 'Fwd Pkt Len Std',
 'Bwd Pkt Len Std',
 'Flow IAT Min',
 'Fwd IAT Tot',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Max',
 'Fwd Header Len',
 'Bwd Header Len',
 'Fwd Pkts/s',
 'Bwd Pkts/s',
 'Pkt Len Min',
 'Pkt Len Max',
 'Pkt Len Mean',
 'Pkt Len Std',
 'SYN Flag Cnt',
 'ECE Flag Cnt',
 'Pkt Size Avg',
 'Fwd Seg Size Avg',
 'Bwd Seg Size Avg',
 'Subflow Fwd Pkts',
 'Subflow Fwd Byts',
 'Subflow Bwd Pkts',
 'Subflow Bwd Byts',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max']

In [43]:
dfCleaned = df.drop(columns=high_corr_features)


In [45]:
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(mapping)

{'Benign': 0, 'FTP-BruteForce': 1, 'SSH-Bruteforce': 2}


In [47]:
dfCleaned.describe()

,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Bwd Pkt Len Max,Bwd Pkt Len Min,...,Bwd Blk Rate Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Idle Min,Label
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,...,1048575.0,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,4.876262e+03,6.255555e+06,6.206622e+00,7.211191e+00,4.479936e+02,1.745736e+02,8.389535e+00,3.879579e+01,3.325241e+02,2.016587e+01,...,0.0,1.050166e+04,6.142295e+03,2.793536e+00,2.327970e+01,5.152449e+04,2.136151e+04,8.789157e+04,2.126920e+06,5.422006e-01
std,1.444344e+04,1.260291e+09,4.447851e+01,1.048682e+02,1.573541e+04,2.876713e+02,1.948279e+01,5.331882e+01,4.938587e+02,4.653823e+01,...,0.0,1.293709e+04,1.814980e+04,5.557106e+00,1.106185e+01,5.815586e+05,2.186405e+05,7.395725e+05,1.817013e+07,7.784714e-01
min,0.000000e+00,-9.190110e+11,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.200000e+01,7.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,2.410000e+02,-1.000000e+00,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.300000e+01,1.023000e+03,2.000000e+00,1.000000e+00,3.600000e+01,3.400000e+01,0.000000e+00,2.566667e+01,5.300000e+01,0.000000e+00,...,0.0,8.192000e+03,0.000000e+00,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,4.066690e+05,7.000000e+00,6.000000e+00,4.550000e+02,1.990000e+02,0.000000e+00,5.550000e+01,5.710000e+02,0.000000e+00,...,0.0,2.688300e+04,2.300000e+02,4.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,6.553300e+04,1.200000e+08,5.115000e+03,9.198000e+03,8.591554e+06,6.444000e+04,1.460000e+03,1.121703e+04,1.460000e+03,7.320000e+02,...,0.0,6.553500e+04,6.553500e+04,1.031000e+03,4.800000e+01,1.102401e+08,5.723446e+07,1.102401e+08,1.260300e+10,2.000000e+00


In [49]:
df_filtered = dfCleaned[dfCleaned['Label'].isin([1, 2])]

numeric_cols = df_filtered.select_dtypes(include=[np.number])

skew_values = numeric_cols.skew()

high_skew_cols = skew_values[abs(skew_values) > 2]
print(high_skew_cols.sort_values(ascending=False))

Bwd IAT Min      21.286208
Down/Up Ratio    -2.384488
dtype: float64


C:\Users\chinm\anaconda3\Lib\site-packages\pandas\core\nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean


In [51]:
df_filtered["Bwd IAT Min"].describe()


count    380949.000000
mean          2.782404
std           6.209008
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         696.000000
Name: Bwd IAT Min, dtype: float64

In [53]:
(df_filtered["Bwd IAT Min"]==0).mean()

0.7537255643143833

In [55]:
dfCleaned["Bwd IAT Min"] = np.log1p(dfCleaned["Bwd IAT Min"].clip(lower=0))

In [57]:
skew=dfCleaned["Bwd IAT Min"].skew()
print(skew)

1.8689535910234154


In [59]:
cols_with_inf = dfCleaned.columns[
    np.isinf(dfCleaned.select_dtypes(include=[np.number])).any()
].tolist()

cols_with_inf

['Flow Byts/s', 'Flow Pkts/s']

In [61]:
dfCleaned.replace([np.inf, -np.inf], np.nan, inplace=True)

In [63]:
dfCleaned.fillna(dfCleaned.median(numeric_only=True), inplace=True)


In [65]:
X = dfCleaned.drop(columns=['Label'])
y = dfCleaned['Label']

In [67]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
df_scaled = pd.concat([X_scaled, y], axis=1)

In [69]:
df_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 47 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  float64
 1   Flow Duration      1048575 non-null  float64
 2   Tot Fwd Pkts       1048575 non-null  float64
 3   Tot Bwd Pkts       1048575 non-null  float64
 4   TotLen Fwd Pkts    1048575 non-null  float64
 5   Fwd Pkt Len Max    1048575 non-null  float64
 6   Fwd Pkt Len Min    1048575 non-null  float64
 7   Fwd Pkt Len Mean   1048575 non-null  float64
 8   Bwd Pkt Len Max    1048575 non-null  float64
 9   Bwd Pkt Len Min    1048575 non-null  float64
 10  Bwd Pkt Len Mean   1048575 non-null  float64
 11  Flow Byts/s        1048575 non-null  float64
 12  Flow Pkts/s        1048575 non-null  float64
 13  Flow IAT Mean      1048575 non-null  float64
 14  Flow IAT Std       1048575 non-null  float64
 15  Flow IAT Max       1048575 non-n

In [73]:
X = df_scaled.iloc[:, 0:46].values
y = df_scaled.iloc[:, 46].values


In [75]:
X.shape

(1048575, 46)

In [77]:
xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    objective='multi:softprob',
    num_class=3,
    tree_method='hist'
)
xgb_model.fit(X, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_class=3, ...)

In [78]:
lgb_model = lgb.LGBMClassifier(
    n_estimators=10,
    num_leaves=128,
    max_depth=-1,
    objective='multiclass',
    num_class=3
)
lgb_model.fit(X, y)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062521 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6662
[LightGBM] [Info] Number of data points in the train set: 1048575, number of used features: 36
[LightGBM] [Info] Start training from score -0.451459
[LightGBM] [Info] Start training from score -1.690634
[LightGBM] [Info] Start training from score -1.720934
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

LGBMClassifier(n_estimators=10, num_class=3, num_leaves=128,
               objective='multiclass')

In [79]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    n_jobs=-1
)
rf_model.fit(X, y)


RandomForestClassifier(max_depth=20, n_estimators=200, n_jobs=-1)

In [80]:
cat_model = CatBoostClassifier(
    iterations=200,
    depth=6,
    learning_rate=0.1,
    loss_function='MultiClass',
    verbose=False
)
cat_model.fit(X, y)


In [81]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X, y)


LogisticRegression(max_iter=1000)

In [82]:
svm_model = SVC(probability=True)
svm_model.fit(X, y)


SVC(probability=True)

In [83]:
class IntrusionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [84]:
class SmallMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(46, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 3)
        )

    def forward(self, x):
        return F.softmax(self.net(x), dim=1)


In [85]:
class DeepMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(46, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 3)
        )

    def forward(self, x):
        return F.softmax(self.net(x), dim=1)


In [86]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = IntrusionDataset(X, y)
loader = DataLoader(dataset, batch_size=4096, shuffle=True)

def train_model(model, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        total_loss = 0
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

    return model


In [87]:
small_mlp = train_model(SmallMLP())
deep_mlp = train_model(DeepMLP())


Epoch 1, Loss: 0.6650
Epoch 2, Loss: 0.5532
Epoch 3, Loss: 0.5520
Epoch 4, Loss: 0.5517
Epoch 5, Loss: 0.5516
Epoch 6, Loss: 0.5516
Epoch 7, Loss: 0.5516
Epoch 8, Loss: 0.5515
Epoch 9, Loss: 0.5515
Epoch 10, Loss: 0.5515
Epoch 1, Loss: 0.6178
Epoch 2, Loss: 0.5517
Epoch 3, Loss: 0.5515
Epoch 4, Loss: 0.5515
Epoch 5, Loss: 0.5515
Epoch 6, Loss: 0.5515
Epoch 7, Loss: 0.5515
Epoch 8, Loss: 0.5515
Epoch 9, Loss: 0.5515
Epoch 10, Loss: 0.5515


In [88]:
expert_outputs = []

expert_outputs.append(xgb_model.predict_proba(X))
expert_outputs.append(lgb_model.predict_proba(X))
expert_outputs.append(rf_model.predict_proba(X))
expert_outputs.append(cat_model.predict_proba(X))
expert_outputs.append(lr_model.predict_proba(X))
expert_outputs.append(svm_model.predict_proba(X))

# Neural outputs
small_mlp.eval()
deep_mlp.eval()

with torch.no_grad():
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    expert_outputs.append(small_mlp(X_tensor).cpu().numpy())
    expert_outputs.append(deep_mlp(X_tensor).cpu().numpy())

# Convert to numpy array
expert_outputs = np.stack(expert_outputs, axis=1)


In [89]:
class GatingNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(46, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 8)
        )

    def forward(self, x):
        return F.softmax(self.net(x), dim=1)


In [90]:
class GatingDataset(Dataset):
    def __init__(self, X, expert_outputs, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.expert_outputs = torch.tensor(expert_outputs, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.expert_outputs[idx], self.y[idx]


In [91]:
gating_dataset = GatingDataset(X, expert_outputs, y)
gating_loader = DataLoader(gating_dataset, batch_size=4096, shuffle=True)

gating_model = GatingNetwork().to(device)
optimizer = torch.optim.Adam(gating_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [92]:

for epoch in range(10):
    total_loss = 0

    for xb, expert_probs, yb in gating_loader:
        xb = xb.to(device)
        expert_probs = expert_probs.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()

        gates = gating_model(xb)  # (batch, 8)
        gates = gates.unsqueeze(2)  # (batch, 8, 1)

        final_output = torch.sum(gates * expert_probs, dim=1)

        loss = criterion(final_output, yb)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Gating Epoch {epoch+1}, Loss: {total_loss/len(gating_loader):.4f}")


Gating Epoch 1, Loss: 0.5527
Gating Epoch 2, Loss: 0.5515
Gating Epoch 3, Loss: 0.5515
Gating Epoch 4, Loss: 0.5515
Gating Epoch 5, Loss: 0.5515
Gating Epoch 6, Loss: 0.5515
Gating Epoch 7, Loss: 0.5515
Gating Epoch 8, Loss: 0.5515
Gating Epoch 9, Loss: 0.5515
Gating Epoch 10, Loss: 0.5515


In [119]:
import joblib

joblib.dump(scaler, "models/scaler.pkl")
joblib.dump(label_encoder, "models/label_encoder.pkl")

# Save classical models
xgb_model.save_model("models/xgb.json")
joblib.dump(lgb_model, "models/lgb.pkl")
joblib.dump(rf_model, "models/rf.pkl")
cat_model.save_model("models/cat.cbm")
joblib.dump(lr_model, "models/lr.pkl")
joblib.dump(svm_model, "models/svm.pkl")

# Save torch models
torch.save(small_mlp.state_dict(), "models/small_mlp.pt")
torch.save(deep_mlp.state_dict(), "models/deep_mlp.pt")
torch.save(gating_model.state_dict(), "models/gating.pt")
